In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
def extract_dataset():
    if os.path.exists('dataAlphanumerik.zip'):
        if not os.path.exists('dataAlphanumerik'):
            with zipfile.ZipFile('dataAlphanumerik.zip', 'r') as zip_ref:
                print("Mengekstrak dataset...")
                zip_ref.extractall('.')
                print("Ekstraksi selesai!")
    else:
        raise FileNotFoundError("File dataAlphanumerik.zip tidak ditemukan!")

extract_dataset()

Mengekstrak dataset...
Ekstraksi selesai!


In [4]:
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

Current working directory: c:\Users\asus\OneDrive\kuliah\semester 5\MBKM\Capstone_Alphanumerik\TRY2


In [5]:
DATA_DIR = os.path.join(current_dir, 'dataAlphanumerik', 'asl_dataset')
print(f"Looking for dataset in: {DATA_DIR}")

Looking for dataset in: c:\Users\asus\OneDrive\kuliah\semester 5\MBKM\Capstone_Alphanumerik\TRY2\dataAlphanumerik\asl_dataset


In [7]:
zip_path = os.path.join(current_dir, 'dataAlphanumerik.zip')
if os.path.exists(zip_path):
    print("Found zip file, extracting...")
    # Buat direktori dataAlphanumerik jika belum ada
    extract_dir = os.path.join(current_dir, 'dataAlphanumerik')
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract ke folder dataAlphanumerik
        zip_ref.extractall(extract_dir)
    print("Extraction complete!")

Found zip file, extracting...
Extraction complete!


In [10]:
def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files[:5]:  # Tampilkan 5 file pertama saja
            print(f'{subindent}{f}')
        if len(files) > 5:
            print(f'{subindent}...')

In [9]:
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Directory {DATA_DIR} tidak ditemukan! Pastikan struktur folder benar.")
else:
    print("\nDataset directory found!")
    # Tampilkan isi folder dataset
    print("\nIsi folder dataset:")
    print_directory_structure(DATA_DIR)


Dataset directory found!

Isi folder dataset:
asl_dataset/
    0/
        hand1_0_bot_seg_1_cropped.jpeg
        hand1_0_bot_seg_2_cropped.jpeg
        hand1_0_bot_seg_3_cropped.jpeg
        hand1_0_bot_seg_4_cropped.jpeg
        hand1_0_bot_seg_5_cropped.jpeg
        ...
    1/
        hand1_1_bot_seg_1_cropped.jpeg
        hand1_1_bot_seg_2_cropped.jpeg
        hand1_1_bot_seg_3_cropped.jpeg
        hand1_1_bot_seg_4_cropped.jpeg
        hand1_1_bot_seg_5_cropped.jpeg
        ...
    2/
        hand1_2_bot_seg_1_cropped.jpeg
        hand1_2_bot_seg_2_cropped.jpeg
        hand1_2_bot_seg_3_cropped.jpeg
        hand1_2_bot_seg_4_cropped.jpeg
        hand1_2_bot_seg_5_cropped.jpeg
        ...
    3/
        hand1_3_bot_seg_1_cropped.jpeg
        hand1_3_bot_seg_2_cropped.jpeg
        hand1_3_bot_seg_3_cropped.jpeg
        hand1_3_bot_seg_4_cropped.jpeg
        hand1_3_bot_seg_5_cropped.jpeg
        ...
    4/
        hand1_4_bot_seg_1_cropped.jpeg
        hand1_4_bot_seg_2_cropped.jpeg

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Ekstrak dataset
def extract_dataset():
    # Pastikan file zip ada
    if os.path.exists('dataAlphanumerik.zip'):
        # Buat direktori jika belum ada
        if not os.path.exists('dataAlphanumerik'):
            with zipfile.ZipFile('dataAlphanumerik.zip', 'r') as zip_ref:
                print("Mengekstrak dataset...")
                zip_ref.extractall('.')
                print("Ekstraksi selesai!")
    else:
        raise FileNotFoundError("File dataAlphanumerik.zip tidak ditemukan!")

# Panggil fungsi ekstrak
extract_dataset()

# Parameter Global
DATA_DIR = './dataAlphanumerik/asl_dataset/'
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Directory {DATA_DIR} tidak ditemukan! Pastikan struktur folder benar.")

# Cek jumlah kelas dan gambar
num_classes = len(os.listdir(DATA_DIR))
total_images = sum([len(files) for r, d, files in os.walk(DATA_DIR)])
print(f"Jumlah kelas yang ditemukan: {num_classes}")
print(f"Total gambar yang ditemukan: {total_images}")

# Konfigurasi parameter
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 40
LEARNING_RATE = 1e-2
VALIDATION_SPLIT = 0.2

SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

# Preprocessing data
rescale_layer = tf.keras.layers.Rescaling(1./255)

# Load dan preprocess dataset
print("Mempersiapkan dataset...")
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    color_mode="rgb"  # MobileNet membutuhkan input RGB
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    color_mode="rgb"
)

print("Dataset berhasil dimuat!")
print(f"Jumlah batch training: {len(train_dataset)}")
print(f"Jumlah batch validasi: {len(val_dataset)}")

# Optimasi performa
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
val_dataset = val_dataset.prefetch(PREFETCH_BUFFER_SIZE)

# Membuat model MobileNet
print("Mempersiapkan model MobileNet...")
base_model = MobileNet(
    weights='imagenet',
    include_top=False,
    input_shape=(64, 64, 3)
)

# Freeze base model
base_model.trainable = False

# Membuat model lengkap
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Jumlah kelas dideteksi otomatis

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model siap untuk training!")
print("\nRingkasan Model:")
model.summary()

# Training model
print("\nMemulai proses training...")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    verbose=1
)

# Simpan model
model.save('asl_mobilenet_model.h5')
print("\nModel telah disimpan sebagai 'asl_mobilenet_model.h5'")

# Plot history training
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()